**Matrix Factorisation:-** It is a mathematical technique commonly used in recommendation systems to factorize a user-item interaction matrix into lower-dimensional matrices representing users and items.


In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1Q9UJtrN_v_dS-garl5gQ1I_SotGhye_1' -O "movies.csv"
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1HOFWUAMFlYbd-gk1B2IyV2-hXDZI7gKR' -O "ratings.csv"
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1b7_yRRBs3s3atp1WQHN2GU577vxY8u_h' -O "users.csv"

--2026-01-15 06:21:44--  https://drive.google.com/uc?export=download&id=1Q9UJtrN_v_dS-garl5gQ1I_SotGhye_1
Resolving drive.google.com (drive.google.com)... 142.250.101.138, 142.250.101.101, 142.250.101.100, ...
Connecting to drive.google.com (drive.google.com)|142.250.101.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1Q9UJtrN_v_dS-garl5gQ1I_SotGhye_1&export=download [following]
--2026-01-15 06:21:44--  https://drive.usercontent.google.com/download?id=1Q9UJtrN_v_dS-garl5gQ1I_SotGhye_1&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.141.132, 2607:f8b0:4023:c03::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.141.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 515699 (504K) [application/octet-stream]
Saving to: ‘movies.csv’

movies.csv          100%[===================>] 503.61K  --.-KB/

In [ ]:
movies=pd.read_csv('/content/movies.csv')
ratings=pd.read_csv('/content/ratings.csv')
users=pd.read_csv('/content/users.csv')

print(movies.shape)
print(ratings.shape)
print(users.shape)

(10329, 3)
(105339, 4)
(668, 3)


In [ ]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
users.head()

,userId,age,time_spent_per_day
0,1,16,3.976315
1,2,24,1.891303
2,3,20,4.521478
3,4,23,2.095284
4,5,35,1.759860


In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [ ]:
ratings=ratings[['userId','movieId','rating']]
ratings.head()

,userId,movieId,rating
0,1,16,4.0
1,1,24,1.5
2,1,32,4.0
3,1,47,4.0
4,1,50,4.0




*   Converting the original user-item interaction data into a user-item matrix where rows represent users, columns represent movies, and the entries contain user ratings.
*   Any missing values (unrated movie) are filled with 0, creating a matri suitable for collaborative filtering in recommendation systems.



In [ ]:
rs=ratings.pivot(index='userId',columns='movieId',values='rating').fillna(0)
rs.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,144482,144656,144976,146344,146656,146684,146878,148238,148626,149532
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Matrix Factorisation From Scratch

In [ ]:
rs_small=rs.iloc[:100,:100]
rs_small.head()
rm_small=rs.copy()
rm_small=rm_small[rm_small.columns[:100]]
rm_small=rm_small.head(100)
# Both code snippets achieve  the same result!!



*   Matrix factorisation using stochastic gradient descent to minimize the cost function.
*   It iteratively optimizes latent  factor matrices P and Q to approximate the user-item interaction matrix R.After optimization, it can predict user-item interactions and compares them with actual ratings.



In [ ]:
k=2 #latent factors
P=np.random.normal(size=(rs_small.shape[0],k))
Q=np.random.normal(size=(rs_small.shape[1],k))
print(P.shape)
print(Q.shape)
print(rs_small.shape)

(100, 2)
(100, 2)
(100, 100)


In [ ]:
def matrix_factorization(R,P,Q,K,steps=10000,alpha=0.0002,beta=0.02):
  Q=Q.T

  for step in range(steps):
    for i in range(len(R)): #loop to all users
      for j in range(len(R[i])): # loop to all movies of that user
        if R[i][j]==0:
          continue #if user has not rated the movie
        else:
          eij=R[i][j]-np.dot(P[i,:],Q[:,j]) #calculate error
          for v in range(k):
            x=P[i][v]
            y=Q[v][j]
            P[i][v]=x+alpha*(2*eij*y)-(beta*x)
            Q[v][j]=y+alpha*(2*eij*x)-(beta*y)
  return P,Q.T


In [ ]:
B,C=matrix_factorization(rs_small.values,P.copy(),Q.copy(),k)
#

In [ ]:
print(B)

[[-2.20742258e-93  2.03755585e-92]
 [-1.61849132e-92  2.35748562e-93]
 [-4.90088710e-93 -2.08167939e-92]
 [ 2.80460987e-89  1.07342815e-89]
 [ 3.89492266e-92 -1.16520332e-92]
 [-4.06576163e-01  3.72263113e-01]
 [-2.76499771e-92  5.09740554e-92]
 [-3.55368958e-93  1.44522638e-93]
 [-9.26270062e-93  5.15154402e-93]
 [ 1.68649463e-89 -1.74266577e-88]
 [-2.25513958e-92 -6.03569306e-93]
 [ 5.72839279e-95 -3.67626553e-94]
 [-4.19101077e-92 -9.50151509e-93]
 [-2.03224214e-92 -6.85459919e-93]
 [ 1.50004710e-92 -5.64637686e-92]
 [-2.55301962e-92  3.23118913e-93]
 [-6.52141728e-93 -4.88033285e-92]
 [-3.05392529e-92  1.26719387e-93]
 [-3.31161446e-01  9.12435496e-01]
 [-6.09417162e-02  6.09805898e-01]
 [ 2.12116260e-88 -7.33488608e-89]
 [-1.42222619e-92  8.40947952e-93]
 [ 4.77045152e-89 -1.10884480e-88]
 [-2.21147095e-92  1.82459313e-92]
 [ 6.09451092e-93 -9.43462694e-93]
 [-2.84560817e-88  1.80652415e-88]
 [ 1.73931496e-88  9.19634507e-89]
 [-8.83082404e-93 -2.34813771e-92]
 [ 2.61809881e-91 -6

In [ ]:
print(C)

[[ 1.00353544e-90 -2.99260482e-91]
 [ 8.24137252e-94 -1.03887559e-92]
 [ 7.95040549e-94 -3.65185995e-94]
 [ 5.57766947e-88 -3.74653496e-88]
 [-1.13480380e-90 -3.18789386e-91]
 [ 1.93273351e-93 -7.53846158e-93]
 [ 1.86119140e-93 -9.45404284e-94]
 [ 1.22429705e+00 -1.23424921e+00]
 [-1.04943680e-92  5.52018797e-92]
 [-1.69252609e-93 -5.80905873e-93]
 [-3.06044889e-93 -7.86317496e-93]
 [-2.15073951e-92  1.41692016e-91]
 [ 1.28675803e+00  5.00353042e-01]
 [-7.72677550e-94  1.29862547e-94]
 [ 1.23982513e-92 -3.03510678e-93]
 [ 4.88752195e-94 -1.18652707e-92]
 [-1.30299124e-90 -3.10761831e-92]
 [ 6.43570649e-88 -2.90631080e-88]
 [-1.08114362e-90 -7.43812023e-91]
 [ 9.89912378e-93 -3.82868124e-93]
 [-4.71509648e-93 -8.26966358e-93]
 [ 7.82887418e-93 -1.33882994e-93]
 [ 5.33158066e-93 -1.53411148e-93]
 [ 1.72973035e-90  9.09651455e-91]
 [-3.62515274e-93 -1.10555836e-92]
 [ 2.32132051e-88 -2.16832735e-88]
 [-1.06236069e+00 -6.30402982e-01]
 [-5.91591861e-93 -2.21068336e-92]
 [ 1.37913184e-92 -2

In [ ]:
#Predicted values vs Actual Ratings
print(np.dot(B[4],C[36]),rs_small.values[4,36])
print(np.dot(B[1],C[0]),rs_small.values[1,0])
print(np.dot(B[1],C[2]),rs_small.values[1,2])
print(np.dot(B[3],C[17]),rs_small.values[3,17])

1.4955224842415613e-182 0.0
-1.6947636247865793e-182 5.0
-1.3728582979288203e-185 2.0
1.4929930114869828e-176 0.0


In [ ]:
from sklearn.metrics import mean_squared_error as mse
rm_=np.dot(B,C.T)
print(mse(rs_small.values,rm_))

0.7639641797403678


In [ ]:
mse(rs_small.values[rs_small>0],rm_[rs_small>0])
#To filter only known ratings from original and predicted matrix

13.10400390625

# Using cmfrec library -> Collective matrix factorization in recommender systems


*   cmfrec library requires input in the form of dataframe not as sparse matrix.
*   It requires 3 columns UserId,ItemId,Rating.

*   It is name specific as well means the name of the columns in dataframe must be  exactlly same as mentioned above.

In [ ]:
ratings.head(2)

,userId,movieId,rating
0,1,16,4.0
1,1,24,1.5


In [ ]:
rs_raw=ratings.copy()
rs_raw.shape

(105339, 3)

In [ ]:
!pip install cmfrec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 9.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached findblas-0.1.26.post1-py3-none-any.whl
  Created wheel for cmfrec: filename=cmfrec-3.5.1.post13-cp312-cp312-linux_x86_64.whl size=5975895 sha256=3a1f1338b4d39692b205df217fae5f5f91707566369814ed3babedf0f8e8257a
  Stored in directory: /root/.cache/pip/wheels/46/9d/a7/03614a1633b40dde9aeb6eb12265f9c684feba1e3d82fe05bf
Successfully built cmfrec


An instance of the CMF model is created with various hyperparameters:

method="als": Specifies the alternating least squares (ALS) optimization method, commonly used for matrix factorization in recommendation systems.

k=2: Sets the number of latent factors to 2, determining the dimensionality of the latent factor space.

lambda_=0.1: Sets the regularization strength to 0.1. Regularization is used to prevent overfitting in the model.

user_bias=False: Indicates that user bias terms are not included in the model. User bias represents a user's overall rating tendency.

item_bias=False: Excludes item bias terms in the model. Item bias represents an item's overall rating tendency.

verbose=False: Suppresses verbose output, controlling whether the model's training progress is displayed.

In [ ]:
from cmfrec import CMF
rs_raw.rename(columns={'userId':'UserId','movieId':'ItemId','rating':'Rating'},inplace=True)

In [ ]:
rs_raw.head()

,UserId,ItemId,Rating
0,1,16,4.0
1,1,24,1.5
2,1,32,4.0
3,1,47,4.0
4,1,50,4.0


In [ ]:
df=rs_raw.copy()
model=CMF(
    method="als",
    k=2,
    lambda_=0.1,
    user_bias=False,
    item_bias=False,
    verbose=False
)

In [ ]:
model.fit(df)

Collective matrix factorization model
(explicit-feedback variant)


In [ ]:
model.A_.shape, model.B_.shape

((668, 2), (10325, 2))

In [ ]:
model.A_

array([[ 0.6181083 , -1.296816  ],
       [ 1.0773844 ,  0.44746563],
       [ 0.7495232 , -0.2906971 ],
       ...,
       [ 0.72334623, -1.0197821 ],
       [ 0.64136606, -0.22247148],
       [-0.2533895 , -1.4885318 ]], dtype=float32)

In [ ]:
rm__=np.dot(model.A_,model.B_.T)
mse(rs.values[rs>0],rm__[rs>0])**0.5 #rmse

3.8516142259951596

## Notebook Summary: Matrix Factorization for Recommendation Systems

This notebook explores the concept of Matrix Factorization, a technique commonly used in recommendation systems, by implementing it from scratch and then utilizing a specialized library (`cmfrec`).

### Objective
The primary goal is to demonstrate and compare two approaches to Matrix Factorization for building a movie recommendation system based on user ratings.

### Data Loading and Preprocessing
1.  **Data Acquisition**: Downloaded `movies.csv`, `ratings.csv`, and `users.csv` datasets from Google Drive.
2.  **Loading Data**: Loaded these CSV files into pandas DataFrames.
3.  **Initial Inspection**: Displayed the shape and head of each DataFrame to understand their structure.
4.  **Ratings Data Preparation**: Filtered the `ratings` DataFrame to include only `userId`, `movieId`, and `rating`.
5.  **User-Item Matrix Creation**: Transformed the `ratings` DataFrame into a user-item interaction matrix (`rs`) using `pivot_table`, where rows represent users, columns represent movies, and values are ratings. Missing values (unrated movies) were filled with zeros.

### Matrix Factorization From Scratch
1.  **Subset Creation**: A smaller subset of the user-item matrix (`rs_small`) (first 100 users, first 100 movies) was created for efficient demonstration of the custom implementation.
2.  **Initialization**: Latent factor matrices `P` (for users) and `Q` (for items) were randomly initialized with `k=2` latent factors.
3.  **Custom `matrix_factorization` Function**: Implemented a `matrix_factorization` function using Stochastic Gradient Descent (SGD) to iteratively optimize `P` and `Q` by minimizing the error between predicted and actual ratings, incorporating regularization (`beta`).
4.  **Evaluation (Custom)**: Calculated the Mean Squared Error (MSE) for the entire predicted matrix and specifically for known (non-zero) ratings, demonstrating the model's performance on the training data.

### Matrix Factorization using `cmfrec` Library
1.  **Library Installation**: The `cmfrec` library was installed.
2.  **Data Preparation for `cmfrec`**: The `ratings` DataFrame was renamed to `UserId`, `ItemId`, and `Rating` to meet the `cmfrec` library's specific input requirements.
3.  **Model Instantiation**: An instance of the `CMF` (Collective Matrix Factorization) model was created with `method="als"` (Alternating Least Squares), `k=2` latent factors, `lambda_=0.1` regularization, and `user_bias=False`, `item_bias=False`.
4.  **Model Training**: The `cmfrec` model was fitted to the prepared `ratings` DataFrame.
5.  **Evaluation (`cmfrec`)**: The RMSE (Root Mean Squared Error) was calculated for the known ratings using the latent factors learned by `cmfrec`, providing a more robust measure of prediction accuracy.